In [1]:
import numpy as np
import pandas as pd

Для временного ряда цен закрытия (Close) ПАО "Сбербанк" (SBER) построить технические индикаторы:

1. индекс относительной силы RSI(14, 20, 80)
2. Полосу Болинжера ВВ(21, 2)

Указание: применить библиотеку TA-lib